# Introduction

In this tutorial, you'll learn how to create interactive maps with the **folium** package.  Along the way, you'll apply your new skills to visualize Boston crime data.

In [ ]:
import pandas as pd
import geopandas as gpd

import folium
from folium import Choropleth, Circle
from folium.plugins import HeatMap

We define a function `embed_map()` for displaying interactive maps.  It accepts two arguments: the variable containing the map, and the name of the HTML file where the map will be saved.

This function ensures that the maps are visible [in all web browsers](https://github.com/python-visualization/folium/issues/812).

In [ ]:
# Function for displaying the map
def embed_map(m, file_name):
    from IPython.display import IFrame
    m.save(file_name)
    return IFrame(file_name, width='100%', height='500px')

# Your first interactive map

We'll begin by creating a relatively simple map with `folium.Map()`.

In [ ]:
# Create a map
m_1 = folium.Map(location=[42.32,-71.0589], tiles='openstreetmap', zoom_start=10)

# Display the map
embed_map(m_1, 'm_1.html')

Several arguments customize the appearance of the map:
- `location` sets the initial center of the map. We use the latitude (42.3601&deg; N) and longitude (-71.0589&deg; E) of the city of Boston.  
- `tiles` changes the styling of the map; in this case, we choose the [OpenStreetMap](https://www.openstreetmap.org/#map=10/42.32/-71.0589) style.  If you're curious, you can find the other options listed [here](https://github.com/python-visualization/folium/tree/master/folium/templates/tiles).
- `zoom_start` sets the initial level of zoom of the map, where higher values zoom in closer to the map.

Feel free to explore by zooming in and out, or by dragging the map in different directions.

# The data

Now, we'll load some crime data to add to the map! 

We won't focus on the data loading step. Instead, you can imagine you are at a point where you already have the data in a pandas DataFrame `crimes`.  The first five rows of the data are shown below.

In [ ]:
#$HIDE_INPUT$
# Load the data
crimes = pd.read_csv("../input/geospatial-course-data/crimes-in-boston/crime.csv", encoding='latin-1')

# Drop rows with missing locations
crimes.dropna(subset=['Lat', 'Long', 'DISTRICT'], inplace=True)

# Focus on major crimes in 2018
crimes = crimes[crimes.OFFENSE_CODE_GROUP.isin([
    'Larceny', 'Auto Theft', 'Robbery', 'Larceny From Motor Vehicle', 'Residential Burglary',
    'Simple Assault', 'Harassment', 'Ballistics', 'Aggravated Assault', 'Other Burglary', 
    'Arson', 'Commercial Burglary', 'HOME INVASION', 'Homicide', 'Criminal Harassment', 
    'Manslaughter'])]
crimes = crimes[crimes.YEAR>=2018]

# Print the first five rows of the table
crimes.head()

# Bubble map

We can create a **bubble map** by looping over rows of the `crimes` DataFrame, and using `folium.Circle()` to add circles to the map.

In [ ]:
# Create a base map
m_2 = folium.Map(location=[42.32,-71.0589], tiles='cartodbpositron', zoom_start=13)

# Add a bubble map to the base map
for i in range(0,len(crimes)):
    Circle(
        location=[crimes.iloc[i]['Lat'], crimes.iloc[i]['Long']],
        radius=3,
        color='darkred',
        fill=True).add_to(m_2)

# Display the map
embed_map(m_2, 'm_2.html')

Note that `folium.Circle()` takes several arguments:
- `location=[crimes.iloc[i]['Lat'], crimes.iloc[i]['Long']]` supplies the location of the crime (in latitude and longitude).
- `radius=3` sets the radius of the circle.  Note that in a traditional bubble map, these are allowed to vary by xyz.
- `color='darkred'` sets the color of each circle.
- `fill=True` ...

In the map above, our bubbles all have uniform size and color.  However, by allowing the `radius` and `color` arguments to vary with each data point, we can 

After creating ..., we use the `add_to()` method to add it to the detailed map of the city.

# Heatmap

To create a heatmap, we use [`folium.plugins.Heatmap()`](https://python-visualization.github.io/folium/plugins.html#folium.plugins.HeatMap).  This shows the density of crime in different areas of the city, where red areas have relatively more criminal incidents.

As we'd expect for a big city, most of the crime happens near the center.

In [ ]:
# Create a base map
m_3 = folium.Map(location=[42.32,-71.0589], tiles='cartodbpositron', zoom_start=12)

# Add a heatmap to the base map
HeatMap(data=crimes[['Lat', 'Long']], radius=10).add_to(m_3)

# Display the map
embed_map(m_3, 'm_3.html')

As you can see in the code cell above, `folium.plugins.HeatMap()` takes a couple arguments:
- `data` is a DataFrame containing the locations that we'd like to plot.  
- `radius` controls the smoothness of the heatmap.  Higher values make the heatmap look smoother (i.e., with fewer gaps).

# Choropleth map

To understand how crime varies by police district, we'll create a choropleth map.

As a first step, we create a GeoDataFrame where each district is assigned a different row, and the "geometry" column contains the geographical boundaries.

In [ ]:
# GeoDataFrame with geographical boundaries of Boston police districts
districts_full = gpd.read_file('../input/geospatial-course-data/Police_Districts/Police_Districts.shp')
districts = districts_full[["DISTRICT", "geometry"]].set_index("DISTRICT")
districts.head()

We also create a Pandas Series called `plot_dict` that shows the number of crimes in each district.

In [ ]:
# Number of crimes in each police district
plot_dict = crimes.DISTRICT.value_counts()
plot_dict.head()

Note that it's very important that `plot_dict` has the same index as `districts` - this is how the code knows how to match the geographical boundaries with appropriate colors.

Using the `folium.Choropleth()` class, we can create a choropleth map.

In [ ]:
# Create a base map
m_4 = folium.Map(location=[42.32,-71.0589], tiles='cartodbpositron', zoom_start=12)

# Add a choropleth map to the base map
Choropleth(geo_data=districts.__geo_interface__, 
           data=plot_dict, 
           key_on="feature.id", 
           fill_color='YlGnBu', 
           legend_name='Major criminal incidents (Jan-Aug 2018)'
          ).add_to(m_4)

# Display the map
embed_map(m_4, 'm_4.html')

Note that `folium.Choropleth()` takes several arguments:
- `geo_data` is a GeoJSON FeatureCollection containing the boundaries of each geographical area.
 - In the code above, we convert the `districts` GeoDataFrame to a [GeoJSON FeatureCollection](https://en.wikipedia.org/wiki/GeoJSON) with the `__geo_interface__` attribute.
- `data` is a Pandas Series containing the values that will be used to color-code each geographical area.  
- `key_on` will always be set to `feature.id`.  
 - This refers to the fact that the GeoDataFrame used for `geo_data` and the Pandas Series provided in `data` have the same index.  To understand the details, we'd have to look more closely at the structure of a GeoJSON Feature Collection (where the value corresponding to the "features" key is a list, where each entry is a dictionary containing an "id" key).
- `fill_color` sets the color scale. 
- `legend_name` labels the legend in the top right corner of the map.

# Your turn

you've learned about a lot of different map styles. put your new skills to use ...